In [1]:
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import requests
import os

In [2]:
r = requests.get("https://play.google.com/store/apps/category/GAME")

soup = BeautifulSoup(r.content, 'html.parser')

categorys = []
for item in soup.find_all("li",{"class":"KZnDLd"}):
    categorys.append(re.search(r'f" href="(.*?)" title="', str(item)).group(1))

updated_categorys = []
for category in categorys:
    if category.startswith("/store/apps/category/GAME") == True:
        updated_categorys.append(category)

links = []
for link in updated_categorys:
    r = requests.("https://play.google.com"+link)
    soup = BeautifulSoup(r.content, 'html.parser')
    sub_links = []
    for a in soup.find_all("div",{"class":"W9yFB"}):
        output = re.search(r'href="(.*?)" jslog=', str(a)).group(1)
        output = output.replace("amp;","")
        sub_links.append(output)
    for sublink in sub_links:
        r = requests.get("https://play.google.com"+sublink,allow_redirects=False)
        soup = BeautifulSoup(r.content, 'html.parser')
        for item in soup.find_all("div",{"class":"b8cIId ReQCgd Q9MA7b"}):
                links.append(re.search(r'href="(.*?)"><div', str(item)).group(1))


In [34]:
data = pd.DataFrame()
icon = 2298
for link in set(links):
    r = requests.get("https://play.google.com"+link)
    soup = BeautifulSoup(r.content, 'html.parser')
    dict = {}
    try:
        dict["Name"] = re.search(r'<span>(.*?)</span>', str(soup.find_all("h1",{"class":"AHFaub"})[0])).group(1)
    except:
        continue
    try:
        dict["Number of Rating"] = re.search(r'class="">(.*?)</span><span', str(soup.find_all("span",{"class":"AYi5wd TBRnV"})[0])).group(1)
    except:
        continue

    try:

        r = requests.get(re.search(r'srcset="(.*?) 2x"', str(soup.find_all("div",{"class":"MSLVtf Q4vdJd"}))).group(1))
        with open(f'c:\\Users\\bjksa\\Desktop\\nöral\\nöral\\video\\{icon}_video.webp', 'wb') as f:
            f.write(r.content)
    except:
        continue

    
    
    r = requests.get(re.search(r'srcset="(.*?) 2x"', str(soup.find_all("div",{"class":"xSyT2c"}))).group(1))
    with open(f'c:\\Users\\bjksa\\Desktop\\nöral\\nöral\\icon\\{icon}_icon.webp', 'wb') as f:
        f.write(r.content)  

    dict["Genre"] = re.search(r'e">(.*?)</a>', str(soup.find_all("a",{"itemprop":"genre"})[0])).group(1)
    dict["Rating"] = re.search(r'-label="Rated (.*?) stars', str(soup.find_all("div",{"class":"pf5lIe"})[0])).group(1)
    dict["Price"] = re.search(r'itemprop="url"/><meta content="(.*?)" itemprop="price"', str(soup.find_all("span",{"class":"oocvOe"})[0])).group(1)


    text = re.search(r'gebd">(.*?)</div>', str(soup.find_all("div",{"jsname":"sngebd"})[0])).group(1)
    text = text.replace("<br/><br/>"," ")
    text = text.replace("<br/>"," ")
    text = text.replace("\t","")
    text = text.replace("</b>"," ")
    text = text.replace("<b>"," ")
    text = text.replace("  "," ")
    text = text.replace("  "," ")
    dict["Description"] = text

    headers = []
    for a in soup.find_all("div",{"class":"BgcNfc"}):
        headers.append(re.search(r'"BgcNfc">(.*?)</div>', str(a)).group(1))
    if 'Eligible for Family Library' in headers:
        headers.remove('Eligible for Family Library')

    items = []
    for a in soup.find_all("span",{"class":"htlgb"}):
        try:
            output = re.search(r'<span class="htlgb"><div class="IQ1z0d"><span class="htlgb">(.*?)</span>', str(a)).group(1)
            output = output.replace("\xa0"," ")
            if output.startswith("<div>"):
                output = re.search(r'<div>(.*?)</div><div><a', output).group(1).replace("</div><div>",",")
            items.append(output)
        except:
            pass

    for header,item in zip(headers,items):
        dict[header] = item 

    dict.pop('Report', None)
    dict.pop('Permissions', None)
    dict.pop('Developer', None)

    CLEANR = re.compile('<.*?>') 
    for item,value in dict.items():
        dict[item] = re.sub(CLEANR, '', value)
        dict[item] = value.replace("amp;","")

    dummy = pd.DataFrame(dict,index=[0])
    data = pd.concat([data,dummy])
    icon += 1 
data = data.reset_index(drop=True)